#### Importing Libraries

In [53]:
from py2neo import Graph
import pandas as pd

#### Integrating with neo4j

In [54]:
graph = Graph('bolt://localhost:7687', auth=('neo4j','sssihms_scl'))

ConnectionUnavailable: Connection has been closed

In [3]:
query ="""
   match(n) return(n)
"""
graph.run(query)

n
"(_736:pname {Count: '933', Name: 'CAGM'})"
"(_737:pname {Count: '924', Name: 'PTCAP'})"
"(_738:pname {Count: '335', Name: 'P103'})"


In [19]:
query ="""
   match(n) detach delete(n)
"""
graph.run(query)

(No data)

In [5]:
def run(query):
    return(graph.run(query).stats())

In [6]:
def index_run(query_list):
    for i in query_list:
        graph.run(i).stats()

In [21]:
query1 ="""CREATE INDEX ON :single_procedures(id)"""

run(query1)

{'indexes_added': 1}

In [68]:
# query = """CREATE CONSTRAINT UniqueCharacterId ON (c:pname) ASSERT c.id IS UNIQUE"""
# run(query)

In [22]:
# query11 ='''DROP index on :single_procedures(id)'''
# run(query11)

In [70]:
# query11 ='''DROP constraint pname'''

# run(query11)

In [23]:
graph.query("call db.schema.visualization")

nodes,relationships
"[(_-1:pname {constraints: [""Constraint( id=5, name='UniqueCharacterId', type='UNIQUENESS', schema=(:pname {id}), ownedIndex=4 )""], indexes: [], name: 'pname'}), (_-2:single_procedures {constraints: [], indexes: ['id'], name: 'single_procedures'})]",[]


## Loading CSV Files

In [24]:
q1 = '''
      load csv with headers from "file:/single_procedures.csv" as row
      MERGE (r:pname{Name:row.Procedures})
          
          SET 
              r.Count = row.count
    '''
run(q1)

{'labels_added': 367, 'nodes_created': 367, 'properties_set': 734}

In [25]:
q2 = '''
        LOAD CSV WITH HEADERS FROM "file:/sssihms_scl.csv" as row
        MATCH (s:pname{Name:row.Procedure1})
        MATCH (t:pname{Name:row.Procedure2})
        merge (s)-[i:NEXT{Sequence_Name:row.Procedures,Count:toInteger(row.count),Support:toFloat(row.Support),Confidence:toFloat(row.Confidence),Lift:toFloat(row.Lift)}]->(t)        
     '''
run(q2)

{'relationships_created': 589, 'properties_set': 2945}

## Page Rank

 Creating In-memory graph

In [31]:
query3 = """CALL gds.graph.create(
              'myGraph',
              'pname',
              'NEXT',
              {
                relationshipProperties:['Lift','Count']    
              }
)"""

run(query3)

{}

In [32]:
query4 = '''CALL gds.pageRank.write.estimate('myGraph', {
          writeProperty: 'pageRank',
          maxIterations: 20,
          dampingFactor: 0.85
        })
        YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory'''

run(query4)


{}

## Page rank

In [34]:
query5 = '''CALL gds.pageRank.stream('myGraph')
        YIELD nodeId, score
        with gds.util.asNode(nodeId).Name as Procedure, score
        return Procedure,score
        ORDER BY score DESC
        '''

df3 = pd.DataFrame(graph.query(query5).to_data_frame()) 
print("\nPAGE RANKING ALGORITHM")
display(df3.head(30))



PAGE RANKING ALGORITHM


,Procedure,score
0,MINREEX,4.970530
1,P2437,4.306431
2,RESUT,4.222734
3,UROPER31,3.840624
4,VPSH,3.556806
5,CAGM,3.244598
6,REEXDE,2.859864
7,BHEVD,2.686954
8,ENTOPER16,2.457795
9,GSOPER35,2.446125


### Droping a in-memory graph

In [29]:
query1= '''CALL gds.graph.drop('myGraph')
        '''
run(query1)


{}

In [ ]:
# query5 = '''CALL gds.degree.stream('myGraph')
#             YIELD nodeId, score
#             RETURN gds.util.asNode(nodeId).Name AS name, score AS followers
#             ORDER BY followers DESC, name DESC
#         '''

# df3 = pd.DataFrame(graph.query(query5).to_data_frame())
# df3

In [ ]:
# query6 = '''CALL gds.eigenvector.stream('myGraph')
#             YIELD nodeId, score
#             RETURN gds.util.asNode(nodeId).Name AS name, score
#             ORDER BY score DESC, name ASC'''

# df7 = pd.DataFrame(graph.query(query6).to_data_frame())
# df7

In [ ]:
# query8 = '''CALL gds.alpha.influenceMaximization.greedy.stream('myGraph', {seedSetSize: 3, concurrency: 4})
#             YIELD nodeId, spread
#             RETURN gds.util.asNode(nodeId).Name AS Name, spread
#             ORDER BY spread ASC'''
# df9 = pd.DataFrame(graph.query(query8).to_data_frame())
# df9

## Closeness centrality

In [47]:
query11 = '''CALL gds.alpha.closeness.harmonic.stream('myGraph', {})
            YIELD nodeId, centrality
            RETURN gds.util.asNode(nodeId).Name AS Procedure, centrality as Centrality
            ORDER BY Centrality DESC
            '''
df11 =  pd.DataFrame(graph.query(query11).to_data_frame())
print("\n CLOSENESS CENTRALITY")
display(df11.head(10))


 CLOSENESS CENTRALITY


,Procedure,Centrality
0,P2437,0.173776
1,MINREEX,0.170574
2,CAGM,0.156359
3,TPI,0.146274
4,RESUT,0.137619
5,31600,0.136236
6,PAGM,0.134414
7,BHEVD,0.132337
8,P2429,0.131227
9,REEXDE,0.130106


## Authorities and Hubs (HITS)

In [51]:
query12 = '''CALL gds.alpha.hits.stream('myGraph', {hitsIterations: 20})
            YIELD nodeId, values
            RETURN gds.util.asNode(nodeId).Name AS Name, round(toFloat(values.auth),5) AS Auth, round(toFloat(values.hub),5) as Hub
            ORDER BY Auth desc

            '''
df12 =  pd.DataFrame(graph.query(query12).to_data_frame())
print("\nProcedures ordered based on \nAuthority scores")

df12.head(10)


Procedures ordered based on 
Authority scores


,Name,Auth,Hub
0,CAGM,0.45317,0.57576
1,P2437,0.31159,0.11674
2,P103,0.23822,0.13868
3,LRHCATH,0.21189,0.40439
4,PAGM,0.20155,0.19416
5,TPI,0.19517,0.17669
6,PTCAP,0.19424,0.21365
7,RFA,0.18484,0.15257
8,ASDDV,0.18050,0.14135
9,EPS,0.17853,0.20594


In [52]:
query12 = '''CALL gds.alpha.hits.stream('myGraph', {hitsIterations: 20})
            YIELD nodeId, values
            RETURN gds.util.asNode(nodeId).Name AS Name, round(toFloat(values.auth),5) AS Auth, round(toFloat(values.hub),5) as Hub
            ORDER BY Hub desc

            '''
df12 =  pd.DataFrame(graph.query(query12).to_data_frame())
print("\nProcedures ordered based on \nHub scores")
df12.head(20)


Procedures ordered based on 
Hub scores


,Name,Auth,Hub
0,CAGM,0.45317,0.57576
1,LRHCATH,0.21189,0.40439
2,PTCAP,0.19424,0.21365
3,EPS,0.17853,0.20594
4,PAGM,0.20155,0.19416
5,PROCATD,0.16367,0.17891
6,TPI,0.19517,0.17669
7,A203,0.13211,0.16047
8,RFA,0.18484,0.15257
9,PVBDL,0.01556,0.14529
